# Analysis Template

Copy this notebook to start a new analysis with consistent setup, cost guardrails, and branded charts.

### Quick start (beginner-friendly)
1. Run the **Setup (run once)** cell.
2. In **Parameters**, choose `DAYS_BACK`.
3. Replace the sample SQL with your own question.
4. Run the remaining cells.

### Links
- GitHub repo: [github.com/aidanm-lla/lla-data](https://github.com/aidanm-lla/lla-data)
- Open this notebook in Colab: [Analysis Template](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/templates/analysis_template.ipynb)

### Other notebooks
- [Search Contribution Overview](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/seo/01_search_contribution_overview.ipynb)
- [Top Pages Search Performance](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/seo/02_top_pages_search_performance.ipynb)
- [Query Drivers by Page](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/seo/03_query_drivers_by_page.ipynb)
- [Opportunity Watchlist](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/seo/04_opportunity_watchlist.ipynb)
- [Top Pages (Last 7 Days)](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/top_pages_last_7_days.ipynb)
- [Traffic Source Quality](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/traffic_sources.ipynb)
- [Time Patterns for Crisis-Related Pages](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/time_patterns.ipynb)
- [Crisis Support Funnel](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/crisis_funnel.ipynb)

In [ ]:
#@title Setup (run once)
import sys
import os

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
    if not os.path.exists("lla-data"):
        !git clone -q https://github.com/aidoanto/lla-data.git
    repo = os.path.abspath("lla-data")
    if repo not in sys.path:
        sys.path.insert(0, repo)
    !pip install -q db-dtypes google-cloud-bigquery kaleido plotly
else:
    for p in ("..", "../.."):
        ap = os.path.abspath(p)
        if ap not in sys.path:
            sys.path.insert(0, ap)

import plotly.express as px

import lifeline_theme
from lla_data import config
from lla_data.bq import build_date_params, default_query_window, dry_run_bytes, get_client, run_query

lifeline_theme.inject_fonts()

client = get_client()

In [ ]:
#@title Parameters
DAYS_BACK = 28 #@param {type:"integer"}

window = default_query_window(DAYS_BACK)

print(f"Project: {config.PROJECT_ID}")
print(f"GA4 dataset: {config.GA4_DATASET}")
print(f"Search Console dataset: {config.SEARCHCONSOLE_DATASET}")
print(f"Window: {window.start_date} to {window.end_date}")

In [ ]:
sql = f"""
SELECT
  report_date,
  SUM(gsc_clicks) AS clicks,
  SUM(gsc_impressions) AS impressions
FROM `{config.PROJECT_ID}.{config.SEARCHCONSOLE_DATASET}.seo_page_daily`
WHERE report_date BETWEEN DATE(@start_date) AND DATE(@end_date)
GROUP BY report_date
ORDER BY report_date
"""

params = build_date_params(window)
estimated_bytes = dry_run_bytes(client, sql, params=params)
print(f"Estimated bytes scanned: {estimated_bytes:,}")

df = run_query(client, sql, params=params)
df.head()

In [ ]:
fig = px.line(df, x="report_date", y="clicks", template="lifeline", title="Example Trend")
lifeline_theme.add_lifeline_logo(fig)
fig.show()